In [70]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
from scipy.stats import norm
from scipy import stats
from scipy.special import ndtri

equity_idx = pd.read_excel('Equity_IDX_universe_list.xlsx')['ID'].tolist()
index_to_replicate = 'HFRIFOF Index'

start_date = datetime.datetime(1999,12,31)
end_date = datetime.datetime(2018,12,31)

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

def bq_res_array_to_pd(res_array):
    a = res_array[0].df().reset_index()
    r_all = pd.DataFrame({'ID':a['ID']})
    for res in res_array:
        r_n = res.df().reset_index()
        r_all = r_all.merge(r_n, on = 'ID')
    return r_all


index_universe = [index_to_replicate] + equity_idx

id_currency = d.crncy()

request = bql.Request(index_universe[0], {'CURRENCY': id_currency})
res = bq.execute(request)
idx_curr = list(res[0].df()['CURRENCY'])

criteria = id_currency == idx_curr[0]
filtered_univ = u.filter(u.list(index_universe), criteria)

request = bql.Request(filtered_univ, d.id())
res = bq.execute(request)
asset_list = res[0].df().reset_index()['ID'].tolist()

price = d.day_to_day_total_return(start = start_date, end = end_date, per = 'M', fill='prev', currency = 'USD')

request = bql.Request(u.list(asset_list), {'Skew': f.skew(price)['value'], 'Kurtosis': f.kurt(price), 'N_data':f.count(price)['value'], 'Max': f.max(price)['value'], 'Min': f.min(price)['value'],
                                           'Mean': f.avg(price)['value'], 'Std': f.std(price)['value']}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue

response = bq.execute(request)
data = bq_res_array_to_pd(response)

r_value_list = []
shapiro_list = []
for asset in asset_list:
    
    min_a = data.loc[data['ID'].isin([asset])]['Min'].values[0] 
    max_a = data.loc[data['ID'].isin([asset])]['Max'].values[0]
    mean = data.loc[data['ID'].isin([asset])]['Mean'].values[0]
    desv = data.loc[data['ID'].isin([asset])]['Std'].values[0]
    n_data = data.loc[data['ID'].isin([asset])]['N_data'].values[0]

    quart1 = ndtri(0.25)
    quart3 = ndtri(0.75)
    quart1 = quart1*desv + mean
    quart3 = quart3*desv + mean

    IQR = quart3 - quart1

    aux_steps = 2*IQR/(n_data**(1/3)) #Freedman–Diaconis rule

    bin_list = list(np.arange(min_a, max_a+aux_steps, aux_steps))
    steps = len(bin_list)
    bin_labels = list(np.arange(0, steps+1))
    bin_labels = [str(lab) for lab in bin_labels]

    bin_labels_dict = dict()
    for i in range(len(bin_list)):
        bin_labels_dict[bin_labels[i]] = bin_list[i]

    bin_data = f.bins(price, bin_list)['value']

    request = bql.Request(asset, {'BINS': bin_data.groupcount(bin_data)}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
    res = bq.execute(request)
    series = res[0].df().reset_index()
    series = series.rename(index=str, columns={series.columns[-2]: 'bin_label'})
    series['bin_val'] = series['bin_label'].map(bin_labels_dict)
    series = series.sort_values(by = ['bin_val'])
    series['pdf_val'] = norm.pdf(stats.zscore(series['bin_val'].tolist()))

    _, _, r_value, _, _ = stats.linregress((series['BINS'].tolist(), series['pdf_val'].tolist()))
    _, shapiro_val = stats.shapiro(series['BINS'].tolist())
    
    r_value_list += [r_value]
    shapiro_list += [shapiro_val]    
    
data['Correl_to_Norm'] = r_value_list
data['Shapiro-Wilk test'] = shapiro_list

data.to_excel('Analisis_DISTRI_NORM_precios.xlsx')
data

ID       DATE  Kurtosis      Mean  N_data       Min       Max  \
0    HFRIFOF Index 2018-12-31  4.611098  0.002973     229 -0.065362  0.068500   
1        SPX Index 2018-12-31  1.141904  0.005090     229 -0.167948  0.109288   
2       INDU Index 2018-12-31  0.959044  0.006170     229 -0.138844  0.108440   
3       CCMP Index 2018-12-31  1.631592  0.005990     229 -0.228861  0.220047   
4       MXEF Index 2018-12-31  1.444224  0.007689     229 -0.273484  0.171872   
5       MXWO Index 2018-12-31  1.586548  0.004550     229 -0.189354  0.113240   
6        NDX Index 2018-12-31  2.036866  0.006422     229 -0.264016  0.249942   
7        RTY Index 2018-12-31  0.768230  0.007476     229 -0.208092  0.165123   
8       MXWD Index 2018-12-31  1.747550  0.004646     229 -0.197914  0.118918   
9       SPXT Index 2018-12-31  1.140387  0.005092     229 -0.167948  0.109292   
10      MXEA Index 2018-12-31  1.447813  0.003958     229 -0.201767  0.129733   
11    NYFANG Index 2018-12-31  0.100593  0.018023      51 -0.125037  0.154640   
12       RAY Index 2018-12-31  1.228836  0.005451     229 -0.177385  0.115093   
13       RIY Index 2018-12-31  1.248804  0.005324     229 -0.174628  0.112085   
14      MXUS Index 2018-12-31  1.159921  0.004994     229 -0.171016  0.109857   
15       RLV Index 2018-12-31  1.849142  0.005846     229 -0.173127  0.122003   
16       MID Index 2018-12-31  1.937205  0.008365     229 -0.217392  0.148698   
17       RLG Index 2018-12-31  1.379809  0.004672     229 -0.176089  0.126474   
18       SML Index 2018-12-31  1.238951  0.009065     229 -0.201455  0.174543   
19       OEX Index 2018-12-31  0.755089  0.004449     229 -0.145966  0.109584   
20      M1EF Index 2018-12-31  1.674152  0.008927     216 -0.273678  0.170866   
21      M1WO Index 2018-12-31  1.582560  0.004082     229 -0.189601  0.112179   
22     MXAPJ Index 2018-12-31  1.540771  0.007691     229 -0.244545  0.152430   
23       RUO Index 2018-12-31  0.885168  0.006454     229 -0.217154  0.232655   
24       RUJ Index 2018-12-31  1.532180  0.008480     229 -0.199786  0.158661   
25       SVX Index 2018-12-31  1.412633  0.005314     229 -0.171072  0.113130   
26       SGX Index 2018-12-31  0.974345  0.004834     229 -0.165128  0.105913   
27       SPR Index 2018-12-31  1.231790  0.005444     229 -0.173208  0.113004   
28      M1WD Index 2018-12-31  1.998208  0.004721     216 -0.198154  0.118042   
29       SPW Index 2018-12-31  2.403894  0.007233     229 -0.211663  0.185649   
..             ...        ...       ...       ...     ...       ...       ...   
33     R2500 Index 2018-12-31  2.271083  0.007842     180 -0.215511  0.154096   
34      MXNA Index 2018-12-31  1.290621  0.005048     229 -0.178186  0.109389   
35       RMC Index 2018-12-31  2.325719  0.007905     229 -0.223595  0.153657   
36      MXKO Index 2018-12-31  1.603525  0.004933     229 -0.193367  0.115701   
37    FTAW01 Index 2018-12-31  1.761480  0.004251     229 -0.198638  0.121278   
38       RUY Index 2018-12-31  0.768230  0.007476     229 -0.208092  0.165123   
39    MXWOSC Index 2018-12-31  2.031853  0.008076     229 -0.226130  0.165870   
40      GSIN Index 2018-12-31  2.049933  0.003293     135 -0.190339  0.114719   
41       RMV Index 2018-12-31  3.624441  0.008400     225 -0.227637  0.166975   
42    MXEFSC Index 2018-12-31  2.925230  0.007206     229 -0.303239  0.230589   
43  M00IWO$O Index 2018-12-31  3.343652  0.005551     229 -0.158602  0.071513   
44    MXWDIM Index 2018-12-31  1.808210  0.004211     229 -0.203177  0.124912   
45    SP5LVI Index 2018-12-31  2.295397  0.006260     229 -0.130064  0.089580   
46       IXM Index 2018-12-31  3.307926  0.004515     229 -0.263099  0.223583   
47      COMP Index 2018-12-31  1.566334  0.006778     229 -0.140581  0.104221   
48      DJGT Index 2018-12-31  0.865651  0.003305     229 -0.154388  0.115704   
49    FTAG01 Index 2018-12-31  1.638864  0.007205     229 -0.278999  0.178492   
50       IXT Index 2018-12-31  2.533362  0.0045